## Importações

In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.naive_bayes import GaussianNB
from scipy.stats import norm
from sklearn.metrics import accuracy_score,classification_report, f1_score, confusion_matrix,recall_score,precision_score
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, f_oneway, f_regression, r_regression
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import numpy as np
from sklearn.naive_bayes import GaussianNB


## Funções Comuns

In [33]:
def funcPlotMatriz(y_test, classificar):
    matriz_de_confusao = confusion_matrix(y_test, classificar)
    sns.heatmap(matriz_de_confusao, cmap='coolwarm', annot=True, linewidth=1, fmt='d')
    plt.show()


def funcMetricas(df_metricas,modelo_Versao, df,Hyper_Parametros,y_test, classificar):
    metricas = [
        modelo_Versao,
        df,
        Hyper_Parametros,
        accuracy_score(y_test, classificar), 
        round(recall_score(y_test, classificar, pos_label=0,average='macro'),2),
        round(precision_score(y_test, classificar, pos_label=0,average='macro'),2),
        round(f1_score(y_test, classificar, pos_label=0,average='macro'),2)
        ]
    
    nova_linha_df = pd.DataFrame([metricas], columns=df_metricas.columns)

    df_metricas = pd.concat([df_metricas, nova_linha_df], ignore_index=True)
    
    return df_metricas


## Extraindo a Base

In [ ]:
df_original = pd.read_excel('../Data Cleaning AED/Base_format.xlsx')

df = df_original.copy()

display(df)

### Dividindo o Dataframe em Valor Explicativo e Resposta

In [ ]:
df_x = df.drop('Status Operacional', axis=1)
df_y = df['Status Operacional']

df_x = pd.DataFrame(df_x)
df_y = pd.DataFrame(df_y)

df_y = pd.DataFrame(df_y)
display(df_x)
display(df_y)

### Transformando os Dados Qualitativos em Quantitativos

In [ ]:
label_encolder = LabelEncoder()

preprocessador = ColumnTransformer(
    transformers=[
        ('categoria', OrdinalEncoder(), make_column_selector(dtype_include=['object','bool']))
    ],
    remainder='passthrough',  # Mantém as outras colunas inalteradas
    verbose_feature_names_out=False,  # Mantém os nomes originais das colunas
    
)

# Ajustar e transformar o DataFrame usando o preprocessador
df_x_encoded = preprocessador.fit_transform(df_x)

df_y_encoded = label_encolder.fit_transform(df_y)

# Se necessário, converter o resultado de volta para um DataFrame, mantendo os nomes das colunas
df_x_encoded = pd.DataFrame(df_x_encoded, columns=preprocessador.get_feature_names_out())
# Matém a ordem do df original
df_x_encoded = df_x_encoded[df_x.columns]

df_y_encoded = pd.DataFrame(df_y_encoded, columns=df_y.columns)

# df_x_encoded=df_x_encoded[df_x.columns]

df_encoded = df_x_encoded.copy()
df_encoded['Status Operacional']=df_y_encoded
df_encoded

display(df_x_encoded)
display(df_y_encoded)

#### Dividindo os DataFrames de Teste e Treino

In [37]:
X_train, X_test, y_train, y_test = train_test_split(df_x_encoded, df_y_encoded, test_size=0.25,random_state=42,shuffle=True)

# Salva o array na memoria para guardar a ordem das colunas
# X_train = np.ascontiguousarray(X_train)
# X_test = np.ascontiguousarray(X_test)

## Modelo GaussianNB

## Teste Simples

In [ ]:
classificador_bayes = GaussianNB()

scores = cross_validate(classificador_bayes, X_train, y_train, cv=5, scoring=['accuracy','precision_macro','recall_macro','f1_macro'])
chaves = ['test_accuracy','test_precision_macro','test_recall_macro','test_f1_macro']

scores_avg = [scores[chave].mean() for chave in chaves] 

print(scores)
print(scores_avg)


In [ ]:
classificador_bayes.fit(X_train, y_train)
classificar = classificador_bayes.predict(X_test)
print(classification_report(y_test,classificar))

In [ ]:
funcPlotMatriz(y_test, classificar)

## Entendendo o Resultado do Modelo

### Distribuição dos Dados

In [ ]:
display(df.columns)

### Correlação das Variáveis

In [ ]:
correlation = df_encoded[df_encoded.columns.to_list()].corr()

sns.heatmap(correlation, annot = True, fmt=".1f", linewidths=.6)
plt.show()

Plotando a matriz de correlação é possivel dizer que as variaveis são independentes, por conterem pouquíssima correlação muita próxima a 0

### Distribuição das Classes Resposta

Iremos verificar a distribuição das classes, por conta do GaussianNB se comportar mal com classes desbalanceadas. Ele tende a priorizar as classes majoritárias e pode ter dificuldade em prever corretamente as classes minoritárias.

In [ ]:
display(df_y_encoded.value_counts())
display(y_train.value_counts())
display(y_test.value_counts())

As classes resposta estão bem balanceadas, sem mostrar grandes disparidades

## Teste do Modelo Embazado Nos Resultados Anteriores e em Obter o Melhor Desempenho Possível do Modelo

### Importando e Tratando Todas as Bases Alterádas em Análise Axploratória

In [44]:
lista_df = [] # Lista para armazenar os dfs

In [ ]:
lista_df.append(df)

df_capping = pd.read_excel('../Data Cleaning AED/Bases/Base_capping.xlsx')
display(df_capping.head(2))
lista_df.append(df_capping)

df_iqr = pd.read_excel('../Data Cleaning AED/Bases/Base_iqr.xlsx')
display(df_iqr.head(2))
lista_df.append(df_iqr)

df_log = pd.read_excel('../Data Cleaning AED/Bases/Base_log.xlsx')
display(df_log.head(2))
lista_df.append(df_log)

df_raiz = pd.read_excel('../Data Cleaning AED/Bases/Base_sqrt.xlsx')
display(df_raiz.head(2))
lista_df.append(df_raiz)


In [ ]:
df_capping.columns

In [47]:
for k,v in enumerate(lista_df):
    ordem_colunas = lista_df[k].columns
    lista_df[k] = preprocessador.fit_transform(v)
    lista_df[k] = pd.DataFrame(lista_df[k], columns=preprocessador.get_feature_names_out())
    lista_df[k] = lista_df[k][ordem_colunas]

# Lista com todos os dataframe


### PIPELINE

In [ ]:
classificador_bayes.get_params()

Esta pipeline será utilizada para, organizar os passos de transformação de dados e para tentar aplicar o melhor modelo possivel nas condições do nosso caso. Além disso, ela também é muito util para replicar o processo em outros modelos. 

### Explicação da Pipe

df_metricas: DataFrame para armazenar o nome do modelo, a base de dados utilizada, as métricas de teste, e a melhor combinação de hiperparâmetros encontrada para cada base de dados.

pipe: Variável que armazena o pipeline completo, contendo todas as etapas de pré-processamento e modelo.

StandardScaler: Centraliza e normaliza os dados, subtraindo a média e dividindo pelo desvio padrão, o que torna a dispersão dos dados mais uniforme.
SelectKBest: Seleciona as variáveis mais importantes, reduzindo a dimensionalidade para facilitar a interpretação e reduzir o número de perguntas aos usuários.
PCA (Principal Component Analysis): Reduz a dimensionalidade após o SelectKBest, preservando a variância máxima.
DecisionTreeClassifier: Modelo de árvore de decisão utilizado após a redução de dimensionalidade.
params_pipe: Dicionário com os parâmetros da pipeline para otimização.

SelectKBest__k e pca__n_components: Definem a quantidade de variáveis a serem mantidas na redução dimensional.
SelectKBest__score_func: Função de avaliação para selecionar as variáveis mais relevantes.
model__criterion, model__max_depth, model__min_samples_split, e model__min_samples_leaf: Hiperparâmetros específicos da árvore de decisão para otimizar a capacidade de generalização.
valores_k: Lista que define o número máximo de variáveis a serem selecionadas (até 10) pelo SelectKBest.

GridSearchCV: Realiza uma busca em grade sobre diferentes combinações de hiperparâmetros na pipeline, aplica validação cruzada e seleciona a melhor combinação com base na acurácia.

# Pipeline

In [ ]:
import pickle
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# DataFrame para armazenar as métricas
df_metricas = pd.DataFrame(columns=['Modelo','DF','Hyper_Parametros','Acuracia','Recall_Media', 'Precision_Media', 'F1_Score_Media'])

# Variáveis para armazenar o melhor modelo e sua acurácia
melhor_acuracia = 0
melhor_classificador = None
melhor_base = None

# Iterando sobre cada DataFrame da lista
for i, data_frame in enumerate(lista_df):
    # Dividindo dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(data_frame[data_frame.columns[:-1]], data_frame['Status Operacional'], test_size=0.25, random_state=42, shuffle=True)
    
    # Pipeline
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('SelectKBest', SelectKBest()),
        ('pca', PCA()),
        ('model', GaussianNB())  # GaussianNB Classifier
    ])
    
    # Hiperparâmetros
    params_pipe = []
    valores_k = range(1, 11)

    for k in valores_k:
        for n in range(1, k + 1):
            params_pipe.append({
                # 'SelectKBest__k': [k],  # Comentei essa linha pois estava comentada no código original
                'pca__n_components': [n],
                'SelectKBest__score_func': [f_classif, f_oneway, f_regression, r_regression],
                'model__var_smoothing': np.logspace(0, -9, num=100)  # Parâmetro ajustado para GaussianNB
            })

    # Busca em grade
    grid_search = GridSearchCV(estimator=pipe, param_grid=params_pipe, n_jobs=-1, scoring='accuracy')

    # Treinando o modelo
    grid_search.fit(X_train, y_train)

    # Melhor modelo encontrado para essa base
    classificador_grafico = grid_search.best_estimator_

    # Avaliação do modelo
    previsao = classificador_grafico.predict(X_test)
    acuracia = grid_search.best_score_

    # Comparar com a melhor acurácia até agora
    if acuracia > melhor_acuracia:
        melhor_acuracia = acuracia
        melhor_classificador = classificador_grafico
        melhor_base = i  # Guardar o índice da melhor base

    # PCA para análise
    pca = classificador_grafico.named_steps['pca']
    kBest = classificador_grafico.named_steps['SelectKBest']
    
    components_df = pd.DataFrame(pca.components_, columns=data_frame.columns[kBest.get_support(indices=True).tolist()])

    # Plotagem da Matriz de Loadings
    plt.figure(figsize=(12, 6))
    sns.heatmap(components_df, cmap='viridis', annot=True)
    plt.xlabel('Características Originais')
    plt.ylabel('Componentes Principais')
    plt.title(f'Matriz de Loadings - Base {i+1}')
    plt.show()

    # Analisando os resultados
    melhor_params = grid_search.best_params_
    metricas = classification_report(y_test, previsao)
    print(f"Melhores parâmetros (Base {i+1}):", melhor_params)
    print(f"Melhor acurácia (Base {i+1}):", acuracia)
    print(metricas)

    funcPlotMatriz(y_test, previsao)  # Presumindo que essa função já esteja implementada

    # Atualizando DataFrame de métricas
    df_metricas = funcMetricas(df_metricas, 'GaussianNB', data_frame, melhor_params, y_test, previsao)

# Exibir as métricas finais
display(df_metricas)

# Exibir qual foi a melhor base
print(f"A melhor base foi a Base {melhor_base+1} com acurácia de {melhor_acuracia}")

# Serializando o melhor modelo (Pipeline completo) da melhor base
with open('melhor_modelo_gaussiannb_base_log.pkl', 'wb') as arquivo:
    pickle.dump(melhor_classificador, arquivo)

### Algumas Conclusões

À primeira vista, a mudança pode parecer pequena, porém, tive que forçar o modelo a trabalhar com no máximo 10 colunas de características, uma redução significativa para um modelo que possuía 16 colunas no treinamento. Mesmo assim, ele conseguiu manter resultados semelhantes no teste simples e, em alguns casos, até melhores.

Outras conclusões que podem ser tiradas são que as colunas selecionadas, que tiveram o melhor desempenho na validação cruzada (Cross-Validation), foram:

- Age
- Height
- Weight
- family_history_with_overweight
- FAVC
- CAEC
- CH20
Por fim, as métricas obtidas pelo modelo não foram das melhores, e é provável que este não seja o melhor modelo para essa base de dados.

In [64]:
df_metricas.to_excel('./Metricas/metricas_bayes.xlsx', sheet_name='bayes_knn', index=False)

In [ ]:
print(melhor_classificador)

In [ ]:
df_metricas.columns